# Emlf Parallel Plate Tutorial  

## Introduction

The steps described in this tutorial, designed for novice users, show how to apply the Electro Static solver to evaluate the electric field between two metallic plates.

The steps are:

- Modeling

  - set up the geometry with built-in primitives, in this case blocks, and basic copy-paste operations.

- Simulation Settings

  - specify the simulation frequency, material properties, and boundary conditions to define the physics of the problem.

  - set up the grid to discretize the model and create the voxels.

- Post Processing

  - extract the results and learn to use different visualization features.

The geometry used is shown in the [create model](#create_model) section.

In [1]:
import sys, os

import s4l_v1
import s4l_v1.document as document
import s4l_v1.model as model
import s4l_v1.simulation.emlf as lf
import s4l_v1.analysis as analysis
import s4l_v1.analysis.viewers as viewers
from s4l_v1.model import Vec3, Translation

import s4l_v1.units as units
from s4l_v1._api.application import run_application, get_app_safe, print_app_info

from z43_scene_viewer import Viewer


WARNING s4l_v1.model 
  File "/home/test/ldev/Simphony/Simphony/PythonAPI/s4l_v1/model/__init__.py", line 47 : Cannot load model.image API. This module might not be part of this distribution. No module named 'ImageModeling'


## Init Project

In [2]:
if get_app_safe() is None:
    run_application()

app = get_app_safe()

if "TEST_PROJECT_OUTPUT_DIR" in os.environ:
    project_dir = os.environ["TEST_PROJECT_OUTPUT_DIR"]
else:
    project_dir = os.path.join(os.getcwd(), "output")

os.makedirs(project_dir, exist_ok=True)    

project_path = os.path.join(project_dir, "tutorial_emlf_parallel_plate.smash")
s4l_v1.document.New()


## Create model
<a id='create_model'></a>

In [3]:
# set modeling unit to cm
model.SetLengthUnits( units.CentiMeter )  # not reflected in GUI!

plate1 = model.CreateSolidBlock( Vec3(0,0,0), Vec3(15,15,0) )
plate1.Name = 'Plate 1'
plate2 = plate1.Clone()
plate2.ApplyTransform( Translation(Vec3(0,0,3)) )
plate2.Name = 'Plate 2'

In [4]:
vw1 = Viewer(image_size=800, interactive=False)
vw1.set_view_direction(Vec3(-1, -1, -1), Vec3(-1,1,0), Vec3(7.5, 7.5, 0), 25)

Image(value=b'', height='800', width='800')

HTML(value='Logger: ')

## Create Simulation

In [5]:
entities = model.AllEntities()

plate1 = entities['Plate 1']
plate2 = entities['Plate 2']

#Setup Settings
sim = lf.ElectroStaticSimulation()
sim.Name = 'Parallel Plates'
sim.SetupSettings.TestFrequency = 1.0, units.kHz

# Materials:
# nothing to be changed for LF

# Sensors
# we keep the default settings

# Boundary Conditions
boundary_plate1 = sim.AddBoundarySettings(plate1)
boundary_plate1.DirichletValue = 1.0
boundary_plate1.Name = 'Dirichlet +1'
boundary_plate2 = sim.AddBoundarySettings(plate2)
boundary_plate2.DirichletValue = -1.0
boundary_plate2.Name = 'Dirichlet -1'


# Grid
global_grid_settings = sim.GlobalGridSettings
global_grid_settings.ManualDiscretization = True
global_grid_settings.MaxStep = (3.0,)*3 # m.u
global_grid_settings.Resolution = (0.075, )*3 #m.u.
global_grid_settings.ManualPadding = True
global_grid_settings.BottomPadding = global_grid_settings.TopPadding = (30., 30., 70.) # m.u.

manual_grid_settings = sim.AddManualGridSettings([plate1,plate2])
manual_grid_settings.MaxStep = (0.63, )*3
manual_grid_settings.Resolution = (0.075, )*3

# Voxels

default_voxel = next(s for s in sim.AllSettings if isinstance(s, lf.AutomaticVoxelerSettings) )
sim.Add(default_voxel, [plate1, plate2])

# Solver settings
sim.SolverSettings.PredefinedTolerances = sim.SolverSettings.PredefinedTolerances.enum.Medium

## Solve

In [6]:
s4l_v1.document.AllSimulations.Add(sim)
sim.UpdateGrid()
sim.CreateVoxels(project_path)

print("starting simulation ...")
sim.RunSimulation(wait=True)
print("done")


starting simulation ...
done


## Analyse

In [7]:
results = sim.Results()

# overall field sensor
overall_field_sensor = results[ 'Overall Field' ]

### E-field distribution

Create a slice viewer for the e field

In [8]:
slice_field_viewer_e_field = viewers.SliceFieldViewer()
slice_field_viewer_e_field.Inputs[0].Connect( overall_field_sensor['EM E(x,y,z,f0)'] )
slice_field_viewer_e_field.Slice.Plane = slice_field_viewer_e_field.Slice.Plane.enum.XZ
slice_field_viewer_e_field.Visualization.Smooth = True
slice_field_viewer_e_field.Update(0)
slice_field_viewer_e_field.GotoMaxSlice()
document.AllAlgorithms.Add( slice_field_viewer_e_field )

In [9]:
vw2 = Viewer(interactive=False)
vw2.set_view_direction(Vec3(0, 1, 0), Vec3(1, 0, 0), Vec3(7.5, 7.5, 0), 70)

Image(value=b'', height='800', width='800')

HTML(value='Logger: ')

In [10]:
from time import sleep
# sleep(1)
vw2.display_event_history()

HTML(value='18:28:50.100018 - Handled: High quality update (@ 5.7 fps)<br>18:28:50.119700 - Handled: High qual…

### Streamline view of the E-field

In [11]:
# Create a streamline viewer for the field
stream_line_viewer_e_field = viewers.StreamLineViewer()
stream_line_viewer_e_field.Inputs[0].Connect( overall_field_sensor['EM E(x,y,z,f0)'] )
stream_line_viewer_e_field.Streamline.NumberIntegrationTimeSteps = 500
stream_line_viewer_e_field.Streamline.IntegrationTime = 0.5
stream_line_viewer_e_field.Streamline.SeedSource = stream_line_viewer_e_field.Streamline.SeedSource.enum.PlaneSource
stream_line_viewer_e_field.Update(0)
document.AllAlgorithms.Add( stream_line_viewer_e_field )

In [12]:
slice_field_viewer_e_field.Visible = False
vw3 = Viewer(image_size=800)
vw3.set_view_direction(Vec3(0, 1, 0), Vec3(1, 0, 0), Vec3(7.5, 7.5, 0), 40)

Image(value=b'', height='800', width='800')

HTML(value='Logger: ')